# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортирование всех необходимых библиотек.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

Чтение датасета, получение информации о таблице. Очевидно, что за целевое значение нужно взять столбец is_ultra, за параметры - все остальное. Также есть очень сильная корреляция столбцом minutes и calls. Удалим calls.

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data.head()) 
data.info()
data.shape
data.corr()
data = data.drop(['calls'], axis=1)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разделение датасета на параметры и целевые значения. 
Далее осуществляется разбивка данных на выборки. Сначала из общей выборки 60% остаются в тренировочной выборке. Затем оставшиеся 40% пополам разбиваются на тренировочную и тестовую. Затем проверяется их размерность. Она соответсвует заданной.

In [3]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']
features_train, features_x, target_train, target_x = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_x, target_x, test_size=0.5, random_state=12345)
print(len(features_train), len(features_valid), len(features_test))
print(len(target_train), len(target_valid), len(target_test))

1928 643 643
1928 643 643


## Исследуйте модели

Строим модели 3 видов: дерево решений, случайный лес и регрессия, затем переибираем гиперпараметры, обучаем модели на тренировочной выборке и тестируем на валидационной выборке:
1. У дерева решений тестируем глубину от 1 до 20. Лучшую точность модели записываем в best_result_1. Лучший показатель 0,79 получился при глубине 3.  
2. У случайного леса тестируем количество характеристик для деления (от 1 до 3), количество деревьев (от 10 до 51 с шагом 10) и глубину деревьев (от 1 до 10). Лучшую точность модели записываем в best_result_2. Лучший показатель 0,81 получился при глубине 8, количестве критериев 2 и количестве деревьев 40.
3. У логической регрессии получили результат 0,71 при максимальном количестве итераций 1000.

Лучший результат у модели случайного леса.

In [4]:
best_model = None
best_result_1 = 0
for depth in range(1, 20):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(features_train,target_train)  
    result = model_1.score(features_valid, target_valid)
    if result > best_result_1:
        best_model_1 = model_1
        best_result_1 = result
        
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result_1, best_model_1)

best_model = None
best_result_2 = 0
for max_features in range(1, 4):
    for est in range(10, 51, 10):
        for depth in range (1, 11):
            model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, max_features=max_features )
            model_2.fit(features_train, target_train) 
            result = model_2.score(features_valid, target_valid) 
            if result > best_result_2:
                best_model_2 = model_2
                best_result_2 = result

print("Accuracy наилучшей модели леса на валидационной выборке:", best_result_2, best_model_2)

model_3 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_3.fit(features_train, target_train) 
best_result_3 = model_3.score(features_valid, target_valid)
print("Accuracy регрессии на валидационной выборке:", best_result_3)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.7884914463452566 DecisionTreeClassifier(max_depth=4, random_state=12345)
Accuracy наилучшей модели леса на валидационной выборке: 0.7993779160186625 RandomForestClassifier(max_depth=8, max_features=1, n_estimators=20,
                       random_state=12345)
Accuracy регрессии на валидационной выборке: 0.7076205287713841


## Проверьте модель на тестовой выборке

Проверяем получившуюся модель случайного леса на тестовой выборке. Результат 0.8, достаточно точный.

In [5]:
print("Accuracy лучшей модели случайного леса на тестовой выборке:", best_model_2.score(features_test, target_test))


Accuracy лучшей модели случайного леса на тестовой выборке: 0.80248833592535


## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность, необходимо обучить новые простые модели, без корректировки гиперпараметров. Accuracy базовой модели случайного леса немного ниже той, что получилась в предыдущем разделе. Следовательно, модель адекватна и работает лучше базовой. Кроме того, видно, насколько сидьно базовая модель дерева решений отстает от обученной нами с глубиной 3.

In [6]:
model = DecisionTreeClassifier(random_state=123)
model.fit(features_train,target_train)  
print("Accuracy базовой модели дерева решений на тестовой выборке:", model.score(features_test, target_test))
model = RandomForestClassifier(random_state=123)
model.fit(features_train,target_train)  
print("Accuracy базовой модели случайного леса на тестовой выборке:", model.score(features_test, target_test))
model = LogisticRegression(random_state=123)
model.fit(features_train, target_train) 
print("Accuracy базовой модели регрессии на тестовой выборке:", model.score(features_test, target_test))


Accuracy базовой модели дерева решений на тестовой выборке: 0.7169517884914464
Accuracy базовой модели случайного леса на тестовой выборке: 0.7978227060653188
Accuracy базовой модели регрессии на тестовой выборке: 0.6982892690513219


### Вывод
Полученный датафрейм был разбит на группы для обучения и тестирования моделей. Были протестированы модели 3 типов, были протестированы разные гиперпараметры. По результатам тестирования, лучшый результат показала модель случайного леса при глубине 8, количестве критериев 2 и количестве деревьев 40. Ее accuracy составила почти 0.81, что ситается хорошим показателем. При этом, Accuracy у модели дерева решений глубины 3 около 0.79, что также является высоким показателем, но при этом она работает значительно быстрее. Можно назвать ее наиболее удачной из протестированных.

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.2 <a class="tocSkip"> </h2>    
    
     
  
    
Могу посоветовать посмотреть видеоразбор <a href="https://www.youtube.com/watch?v=YTLDATQLK6w">Как обучить модель по шагам в Data Science?</a>
    
Рекомендую тебе ознакомиться с  `GridSearchCV` для подбора гиперпараметров, это избавит тебя от написания циклов. Полезные ссылки: https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv
        
https://newtechaudit.ru/gridsearchcv-pomoshhnik-v-vybore/        
    
    
  
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
